## **Sessa Empirical Estimator**  
### *By Jyreneah Angel and Nicole Grace Joligon* 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import ecdf

In [3]:
# Simulated dataset based on med.events structure
data = pd.DataFrame({
    'pnr': np.repeat(np.arange(1, 11), 5),  # 10 patients
    'eksd': pd.date_range(start='2025-01-01', periods=50, freq='7D'),  # Prescription dates
    'perday': np.random.randint(1, 4, size=50),  # Dosage per day
    'ATC': np.random.choice(['C09CA01', 'C07AB02'], size=50),  # Drug classification
    'dur_original': np.random.randint(10, 30, size=50)  # Prescribed duration
})

def see_algorithm(data, drug_name):
    drug_data = data[data['ATC'] == drug_name].copy()
    drug_data.sort_values(by=['pnr', 'eksd'], inplace=True)
    drug_data['prev_eksd'] = drug_data.groupby('pnr')['eksd'].shift(1)
    drug_data = drug_data.dropna()
    drug_data['event_interval'] = (drug_data['eksd'] - drug_data['prev_eksd']).dt.days
    
    # Compute ECDF & Retain 80%
    ecdf_values = np.sort(drug_data['event_interval'])
    threshold = np.percentile(ecdf_values, 80)
    filtered_data = drug_data[drug_data['event_interval'] <= threshold]
    
    # Log transformation and clustering
    filtered_data['log_interval'] = np.log(filtered_data['event_interval'])
    scaler = StandardScaler()
    scaled_intervals = scaler.fit_transform(filtered_data[['log_interval']])
    
    # Determine optimal k using silhouette (skipping for simplicity in this version)
    optimal_k = 3  # Placeholder, should use silhouette method
    kmeans = KMeans(n_clusters=optimal_k, random_state=42).fit(scaled_intervals)
    filtered_data['Cluster'] = kmeans.labels_
    
    # Assign Median duration per cluster
    cluster_medians = filtered_data.groupby('Cluster')['event_interval'].median()
    filtered_data['Predicted_Duration'] = filtered_data['Cluster'].map(cluster_medians)
    
    return filtered_data[['pnr', 'eksd', 'Predicted_Duration', 'Cluster']]

# Run for both medications
medA_results = see_algorithm(data, 'C09CA01')
medB_results = see_algorithm(data, 'C07AB02')

# Visualization of duration assumptions
def visualize_assumptions(results, drug_name):
    plt.figure(figsize=(10, 5))
    sns.boxplot(x='Cluster', y='Predicted_Duration', data=results)
    plt.title(f'Duration Assumption for {drug_name}')
    plt.show()

visualize_assumptions(medA_results, 'C09CA01')
visualize_assumptions(medB_results, 'C07AB02')


Error: Please load a database to perform operations